In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from pathlib import Path
from collections import Counter

from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# Read the CSV and Perform Basic Data Cleaning

In [6]:
# Load the data
file_path = "c:./Resources/Open_Closed_Charged 082921.csv"
df = pd.read_csv(file_path)
df.head()

,Account Number,Account Type,Loan ID,Loan Open Date,Loan Close Date,Loan Chargeoff Date,Loan\nType,Loan Type Description,Loan Branch,Branch Name,...,Interest Type,Interest Due,Interest From Open,Credit Limit,Exposure,DQ Days,Cr Pmt Current,Credit Score,RBL Tier,Insurance Type
0,178627,0,0,7/27/2007,12/17/2012,NaN,5,USED AUTO L5,21,Loan Administration,...,0,$0.00,"$2,694.31",$0,$0,0,0,0,3,90
1,257568,0,2,12/21/2013,NaN,1/27/2016,9,PERSONAL L9,1,Main Branch,...,0,$1.47,$516.45,$0,"$2,642",2201,L,670,3,90
2,285937,0,0,1/1/1901,NaN,1/1/1901,99,CHARGE OFF LOAN L99,16,IEG Branch,...,0,$0.00,$0.00,$0,"$144,952",0,B,0,NaN,90
3,12121,0,0,12/21/1979,11/15/2002,9/19/2001,99,CHARGE OFF LOAN L99,1,Main Branch,...,0,$0.00,$871.11,$0,$0,0,D,0,NaN,0
4,15335,0,0,1/23/1982,NaN,4/26/2000,99,CHARGE OFF LOAN L99,1,Main Branch,...,0,$0.00,$77.63,$0,$410,7979,D,0,NaN,0


In [4]:
# Find null values
for column in df.columns:
    print(f'Column {column} has {df[column].isnull().sum()} null values')

Column   Account Number has 0 null values
Column Account Type has 0 null values
Column Loan ID has 0 null values
Column Loan Open Date has 0 null values
Column Loan Close Date has 39123 null values
Column Loan Chargeoff Date has 50649 null values
Column Loan
Type has 0 null values
Column Loan Type Description has 0 null values
Column Loan Branch has 0 null values
Column Branch Name has 0 null values
Column  Purpose Code has 0 null values
Column Purpose Code Desc has 344 null values
Column Open Date has 0 null values
Column Collateral Code has 0 null values
Column Collateral Code Desc has 1 null values
Column Interest Rate has 0 null values
Column Original Balance has 0 null values
Column Current Balance has 0 null values
Column Interest Type has 0 null values
Column Interest Due has 0 null values
Column Interest From Open has 0 null values
Column Credit Limit has 0 null values
Column Exposure has 0 null values
Column DQ Days has 0 null values
Column Cr Pmt Current has 8 null values
Col

In [ ]:
# Create a new DataFrame that removes useless columns.

crypto_names_df = crypto_df.drop(columns=["Branch Name", 
                                          "xxxx", 
                                          "xxxx", 
                                          "xxxx"])
crypto_names_df

In [ ]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

In [ ]:
list(df.select_dtypes(include='object').columns)

In [ ]:
# Binary encoding using Pandas # pd.get_dummies
loans_binary_encoded = pd.get_dummies(df, columns=['Loan Type Description',
#                                                    'verification_status',
#                                                    'issue_d',
#                                                    'pymnt_plan',
#                                                    'initial_list_status',
#                                                    'next_pymnt_d',
#                                                    'application_type',
#                                                    'hardship_flag',
#                                                    'debt_settlement_flag'])
loans_binary_encoded.head()

# Split the Data into Training and Testing

In [ ]:
# Create our features
X = loans_binary_encoded.copy()
X = X.drop('loan_status', axis=1)
X.head()

In [ ]:
# Create our target
y = df['loan_status']
y[:5]

In [ ]:
X.describe()

In [ ]:
# Check the balance of our target values
# y['loan_status'].value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

# Scale

In [ ]:
# # Creating a StandardScaler instance.
# scaler = StandardScaler()
# # Fitting the Standard Scaler with the training data.
# X_scaler = scaler.fit(X_train)

# # Scaling the data.
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
# Create a random forest classifier.
rf_model = BalancedRandomForestClassifier(n_estimators=128, random_state=1) 

# Fitting the model
rf_model = rf_model.fit(X_train, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test)


In [ ]:
predictions

In [ ]:
balanced_accuracy_score(y_test, predictions)

In [ ]:
# Display the confusion matrix
predictions = rf_model.predict(X_test)
confusion_matrix(y_test, predictions)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

# Sort by feature importance

In [ ]:
# List the features sorted in descending order by feature importance
# loans_binary_encoded.columns.tolist()

sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

### Easy Ensemble AdaBoost Classifier

In [ ]:
# Train the EasyEnsembleClassifier


# Create a random forest classifier.
eb_model = EasyEnsembleClassifier(n_estimators=128, random_state=1) 

# Fitting the model
eb_model = eb_model.fit(X_train, y_train)

# Making predictions using the testing data.
predictions = eb_model.predict(X_test)

In [ ]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, predictions)

In [ ]:
# Display the confusion matrix
predictions = eb_model.predict(X_test)
confusion_matrix(y_test, predictions)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

In [ ]:
df.columns.tolist()